In [63]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
import requests 
from time import sleep

conn = sqlite3.connect('data/movies.db')
cur = conn.cursor()

In [64]:
imdb_title_basics = pd.DataFrame(cur.execute("SELECT tconst, primary_title, genres FROM imdb_title_basics").fetchall())
imdb_title_basics.columns = [i[0] for i in cur.description]

In [65]:
imdb_title_basics.dropna(inplace=True)

In [66]:
#reformat genres data go make it a list of items
def get_unique_genres(raw_genres):
    all_genres = []
    genres = raw_genres.str.replace("&", ",")
    genres = raw_genres.str.split(',')
    for item in genres:
        if item != None:
            for genre in item:
                all_genres.append(genre)
    unique_genres = set(all_genres)
    unique_genres = set([(genre.strip()) for genre in unique_genres])
    return unique_genres

In [67]:
unique_genres = get_unique_genres(imdb_title_basics["genres"])

In [68]:
#function makes new databse with just a certain column
def get_genre_db(db, genre):
    return db[db["genres"].str.contains(genre)]

In [69]:
action = get_genre_db(imdb_title_basics, "Action")

In [77]:
#sc
action.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10335 entries, 0 to 146128
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tconst         10335 non-null  object
 1   primary_title  10335 non-null  object
 2   genres         10335 non-null  object
dtypes: object(3)
memory usage: 323.0+ KB


In [70]:
#put dataframe that includes tconst and primary title as parameter and returns 
def imdb_page(tconst):
        #tconst = imdb_data["tconst"][x]
    url = f'https://www.imdb.com/title/{tconst}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        soupy = soup.find('div', {'data-testid': 'title-boxoffice-section'})\
        .find_all('li', {'role': 'presentation'})
        return soupy
    except:
        pass

In [72]:
def get_box_office(element, soup):
    if soup != None:
        for x in soup:
            try:
                if (x.find('span', {'class': 'ipc-metadata-list-item__label'}).text == element):
                    ammount = x.find('span', {'class': 'ipc-metadata-list-item__list-content-item'}).text
                    return ammount
            except:
                pass

In [73]:
def get_all_box_office(element, data):
    tconst_list = []
    count = 0
    for tconst in data['tconst']:
        if count == 4000:
            sleep(60* 5)
            count = 0
        tconst_list.append(get_box_office(element, imdb_page(tconst)))
        count += 1
    data[element] = tconst_list

In [74]:
get_all_box_office("Gross worldwide", action)

In [75]:
#sc
imdb_title_basics.head()

,tconst,primary_title,genres
0,tt0063540,Sunghursh,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,Drama
3,tt0069204,Sabse Bada Sukh,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,"Comedy,Drama,Fantasy"


In [76]:
imdb_title_bascis.dropna(subset=['Gross worldwide'])

NameError: name 'imdb_title_bascis' is not defined